In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import MultiHeadAttention

In [ ]:
# Step 1: Data Preparation
# You'll need a large corpus of text data for training
corpus = [
    'The cat sat on the mat',
    'The dog ate my homework',
    'I love deep learning',
    # Add more text data here...
]


In [ ]:
# Step 2: Text Preprocessing
def preprocess_text(corpus):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)

    return predictors, label, total_words, max_sequence_len, tokenizer

In [ ]:
predictors, label, total_words, max_sequence_len, tokenizer = preprocess_text(corpus)

In [ ]:
# Step 3: Define the Model
def create_model(total_words, max_sequence_len):
    embedding_dim = 100
    num_heads = 2
    ff_dim = 64

    inputs = tf.keras.layers.Input(shape=(max_sequence_len-1,))
    embedding_layer = tf.keras.layers.Embedding(total_words, embedding_dim)(inputs)

    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedding_layer, embedding_layer)
    attention_output = tf.keras.layers.Dropout(0.1)(attention_output)
    attention_output = tf.keras.layers.Add()([embedding_layer, attention_output])
    attention_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output)

    lstm_output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True))(attention_output)
    lstm_output = tf.keras.layers.Dropout(0.2)(lstm_output)

    flatten = tf.keras.layers.Flatten()(lstm_output)
    dense_layer = tf.keras.layers.Dense(ff_dim, activation='relu')(flatten)
    output_layer = tf.keras.layers.Dense(total_words, activation='softmax')(dense_layer)

    model = tf.keras.models.Model(inputs=inputs, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


In [ ]:
model = create_model(total_words, max_sequence_len)


In [ ]:
# Step 4: Train the Model
history = model.fit(predictors, label, epochs=100, verbose=1)

In [ ]:
# Step 5: Generate Text
def generate_text(seed_text, next_words, max_sequence_len, model, tokenizer):
    generated_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Sample the word index from the predicted probabilities
        predicted_index = np.random.choice(len(predicted_probs), p=predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
        generated_text += " " + output_word
    return generated_text

In [ ]:
# Example usage:
seed_text = "cat"
generated_text = generate_text(seed_text, 5, max_sequence_len, model, tokenizer)
print(f"Generated Text: {generated_text}")

In [ ]:
# Example usage:
seed_text = "make"
generated_text = generate_text(seed_text, 5, max_sequence_len, model, tokenizer)
print(f"Generated Text: {generated_text}")